# **Dependancies**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import *
from keras.callbacks import *
import os
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from commons import mean_absolute_percentage_error
from keras.layers import *
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


# **Loading Data**

In [ ]:
selected = pd.read_csv("../Data/train_btc_selected_features.csv")
btc = pd.read_csv("../Data/btc_Data.csv")
btc['Date'] = pd.to_datetime(btc['Date'])
btc = btc.set_index("Date")

In [ ]:
btcData = btc[selected.columns]
btcData['returns'] = btcData['priceUSD'].pct_change().copy()
Data = btcData.drop(columns=['priceUSD'])
Data = Data[1:]
# divide X and Y
X = Data.iloc[:,0:]
#Y = Data['returns']   # 用returns的话就用这一行，然后把下一行comment掉
Y = btcData['priceUSD'].shift(-30)[1:] # 反之亦然
# Split into three data sets
X_train = X['2016-01-01':'2019-12-31']
X_val = X['2020-01-01':'2021-05-31']
X_test = X['2021-06-01':'2023-01-01']

Y_train = Y['2016-01-01':'2019-12-31']
Y_val = Y['2020-01-01':'2021-05-31']
Y_test = Y['2021-06-01':'2023-01-01']

In [9]:
estimators=[]

In [10]:
estimators.append(['robust',RobustScaler()])

In [11]:
estimators.append(['mixmax',MinMaxScaler()])

In [12]:
scale=Pipeline(estimators,verbose=True)

In [13]:
X_train=scale.fit_transform(X_train)

[Pipeline] ............ (step 1 of 2) Processing robust, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing mixmax, total=   0.0s


In [14]:
X_test=scale.transform(X_test)

In [15]:
X_train=np.reshape(X_train,(X_train.shape[0],1,X_train.shape[1]))

In [16]:
X_test=np.reshape(X_test,(X_test.shape[0],1,X_test.shape[1]))

In [17]:
y_train=y_train.values

In [18]:
y_train=np.reshape(y_train, (y_train.shape[0],1,1))

In [19]:
y_test=y_test.values

In [20]:
y_test=np.reshape(y_test,(y_test.shape[0],1,1))

# **Model Architecture + Training**

In [22]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [23]:
adam=optimizers.adam(lr=lr_schedule(0),amsgrad=True)


Learning rate:  0.001


In [24]:
model = Sequential()
model.add(Bidirectional(LSTM(350, return_sequences=True, activation='relu'), input_shape=(1, X_train.shape[2])))
model.add(Bidirectional(LSTM(350, return_sequences=True, activation='relu')))
model.add(Dense(1))
model.compile(loss="logcosh", optimizer=adam, metrics=['mae'])

In [25]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, mode='auto')

model.fit(X_train,y_train, epochs=5000, batch_size=32, validation_data=(X_test,y_test), callbacks=[mcp_save,earlyStopping])

# **Testing Model**

In [27]:
from tensorflow.keras.models import load_model
prediction_model = load_model('trained_models/LSTM_reg_seven_new.hdf5',compile=False)

In [28]:
y_pred = np.ravel(prediction_model.predict(X_test))

In [29]:
y_test=np.ravel(y_test)

In [30]:
r2=r2_score(y_test,y_pred) #testing score/ r^2
r2

0.9995161692423405

In [31]:
mae=mean_absolute_error(y_test,y_pred) #mae
mae

35.716042185424314

In [32]:
rmse=np.sqrt(mean_squared_error(y_test,y_pred)) #rmse
rmse

83.31697071794044

In [33]:
mape=mean_absolute_percentage_error(y_test,y_pred) #mape
mape

1.5003317888342855

In [34]:
pd.DataFrame(zip(['MAE','RMSE','MAPE','R^2'],[mae,rmse,mape,r2])).transpose()

,0,1,2,3
0,MAE,RMSE,MAPE,R^2
1,35.716,83.317,1.50033,0.999516


In [35]:
pd.DataFrame([y_test,y_pred]).transpose()

,0,1
0,6912.000,6930.309570
1,6233.000,6215.681641
2,235.065,236.442825
3,609.126,614.928223
4,453.641,455.907837
5,852.382,832.241394
6,342.284,338.540771
7,344.899,347.315247
8,644.487,628.036255
9,3804.000,3817.390625
